# Extracción de datos desde bases de datos

## Conexión a bases de datos relacionales con ODBC y SQLAlchemy

#### Tipos de módulos de conexión
Para desarrollar un proyecto en Python en el que se quiera implementar el uso "Bases de Datos SQL", se tienen distintos módulos que pueden ser útiles, por ejemplo "PYODBC" que es uno de ellos.

**ODBC:**

Open DataBase Connectivity (ODBC) es un estándar de acceso a las bases de datos desarrollado por SQL Access Group (SAG) en 1992. El objetivo de ODBC es hacer posible el acceder a cualquier dato desde cualquier aplicación, sin importar qué sistema de gestión de bases de datos (DBMS) almacené los datos.

**PYODBC:**

Pyodbc es un módulo de Python de código abierto que simplifica el acceso a las bases de datos "ODBC" desde Python, implementando el uso de la DB API 2.0 de una forma conveniente para Python. Pyodbc también es considerado como un controlador SQL para Python.

### Conexión a bases de datos SQL Server utilizando cursor

#### Configuracion de SQL Server

So just open the access to your 127.0.0.1:1433 in the SQL server Configuration Manager.

Steps:

- Start -> All Programs -> SQL Server Configuration Manager
- SQL Server Network Configuration -> Protocols for MSSQLSERVER -> TCP/IP (Enable it)
- TCP/IP -> Properties -> IP Addresses. Find 127.0.0.1 and change the "Enabled" to "Yes". You can do it for all the IPs if you want.

In [ ]:
! conda install pymssql -y

In [4]:
import pymssql

conn = pymssql.connect(
    server='DSANDOVALFLAVIO', 
    database='cf_etl'
)

In [6]:
# Crear el cursor y ver las tablas de la base de datos
cursor = conn.cursor()
cursor.execute('SELECT * FROM information_schema.tables')
cursor.fetchall()

[('cf_etl', 'dbo', 'dataset', 'BASE TABLE')]

In [10]:
query = 'SELECT top 3 * FROM cf_etl.dbo.dataset'
cursor.execute(query)
data = cursor.fetchall()
data

[('12/24/2023',
  'CursosDeProgramacion',
  'Oeste',
  26148.0,
  133468.0,
  21979.0,
  606.0),
 ('11/11/2023',
  'AprendeCSharpFacil',
  'Norte',
  34586.0,
  340290.0,
  49146.0,
  495.0),
 ('5/24/2024',
  'CodigoFacilitoEnEspanol',
  'Norte',
  20337.0,
  343710.0,
  32831.0,
  1655.0)]

### Conexión a bases de datos SQL Server utilizando ODBC y SQLAlchemy

In [ ]:
! conda install pyodbc -y

In [14]:
! conda install sqlalchemy -y

Channels:
 - defaults
 - conda-forge
Platform: win-64
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\flavi\anaconda3\envs\tch_cf_etl_env

  added / updated specs:
    - sqlalchemy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    greenlet-3.0.1             |  py312hd77b12b_0         219 KB
    sqlalchemy-2.0.30          |  py312h827c3e9_0         3.8 MB
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will be INSTALLED:

  greenlet           pkgs/main/win-64::greenlet-3.0.1-py312hd77b12b_0 
  sqlalchemy         pkgs/main/win-64::sqlalchemy-2.0.30-py312h827c3e9_0 



Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


Drivers Disponibles

In [15]:
import pyodbc

pyodbc.drivers()

['SQL Server',
 'ODBC Driver 17 for SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)']

In [16]:
from sqlalchemy import create_engine

driver = 'ODBC Driver 17 for SQL Server'
server = 'DSANDOVALFLAVIO'
database = 'cf_etl'

engine = create_engine(f'mssql+pyodbc://{server}/{database}?driver={driver}')

# Testear la conexión
try:
    conn = engine.connect()
    print('Conexión exitosa')
except Exception as e:
    print(f'Error en la conexión: {e}')

Conexión exitosa


In [22]:
import pandas as pd

df_table = pd.read_sql_table('dataset', engine)
df_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Fecha        1000 non-null   object 
 1   sCampaign    1000 non-null   object 
 2   Region       1000 non-null   object 
 3   Clicks       1000 non-null   float64
 4   Impresiones  1000 non-null   float64
 5   Views        1000 non-null   float64
 6   Costo        1000 non-null   float64
dtypes: float64(4), object(3)
memory usage: 54.8+ KB


### Ejecución de consultas SQL para extraer datos específicos

In [25]:
query = """
SELECT * 
FROM cf_etl.dbo.dataset 
WHERE sCampaign = 'CursosDeProgramacion'
"""

df = pd.read_sql(
    sql=query, 
    con=engine,
    
    )
df

,Fecha,sCampaign,Region,Clicks,Impresiones,Views,Costo
0,12/24/2023,CursosDeProgramacion,Oeste,26148.0,133468.0,21979.0,606.0
1,3/12/2024,CursosDeProgramacion,Oeste,32254.0,206296.0,6448.0,726.0
2,3/20/2024,CursosDeProgramacion,Norte,7761.0,178949.0,47475.0,862.0
3,10/22/2023,CursosDeProgramacion,Este,42967.0,356762.0,7496.0,1722.0
4,4/22/2024,CursosDeProgramacion,Norte,13362.0,157531.0,47034.0,517.0
...,...,...,...,...,...,...,...
92,4/1/2024,CursosDeProgramacion,Oeste,29805.0,381302.0,19833.0,1152.0
93,5/12/2024,CursosDeProgramacion,Sur,49996.0,79199.0,38665.0,1603.0
94,5/7/2024,CursosDeProgramacion,Este,22068.0,435276.0,34668.0,577.0
95,10/31/2023,CursosDeProgramacion,Este,31943.0,248545.0,21555.0,882.0


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Fecha        97 non-null     object 
 1   sCampaign    97 non-null     object 
 2   Region       97 non-null     object 
 3   Clicks       97 non-null     float64
 4   Impresiones  97 non-null     float64
 5   Views        97 non-null     float64
 6   Costo        97 non-null     float64
dtypes: float64(4), object(3)
memory usage: 5.4+ KB


In [27]:
df = pd.read_sql(
    sql=query, 
    con=engine,
    parse_dates={'Fecha': {'format': '%m-%d-%Y'}}
    )
df

,Fecha,sCampaign,Region,Clicks,Impresiones,Views,Costo
0,12/24/2023,CursosDeProgramacion,Oeste,26148.0,133468.0,21979.0,606.0
1,3/12/2024,CursosDeProgramacion,Oeste,32254.0,206296.0,6448.0,726.0
2,3/20/2024,CursosDeProgramacion,Norte,7761.0,178949.0,47475.0,862.0
3,10/22/2023,CursosDeProgramacion,Este,42967.0,356762.0,7496.0,1722.0
4,4/22/2024,CursosDeProgramacion,Norte,13362.0,157531.0,47034.0,517.0
...,...,...,...,...,...,...,...
92,4/1/2024,CursosDeProgramacion,Oeste,29805.0,381302.0,19833.0,1152.0
93,5/12/2024,CursosDeProgramacion,Sur,49996.0,79199.0,38665.0,1603.0
94,5/7/2024,CursosDeProgramacion,Este,22068.0,435276.0,34668.0,577.0
95,10/31/2023,CursosDeProgramacion,Este,31943.0,248545.0,21555.0,882.0


# Extracción de datos desde bases de datos NoSQL (MongoDB)

In [28]:
! conda install pymongo -y

Channels:
 - defaults
 - conda-forge
Platform: win-64
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\flavi\anaconda3\envs\tch_cf_etl_env

  added / updated specs:
    - pymongo


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cryptography-43.0.0        |  py312h89fc84f_0         1.4 MB
    dnspython-2.4.2            |  py312haa95532_0         480 KB
    pymongo-4.6.3              |  py312hd77b12b_0         1.2 MB
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  cryptography       pkgs/main/win-64::cryptography-43.0.0-py312h89fc84f_0 
  dnspython          pkgs/main/win-64::dnspython-2.4.2-py312haa95532_0 
  pymongo            pkgs/main/win-64::pymongo-4.6.3-py312hd77b12b_0 



Preparing transaction: ...working...

In [35]:
import pymongo

client = pymongo.MongoClient('mongodb://localhost:27017/')
mydb = client['cf_etl']
mycollection = mydb['dataset']

In [36]:
#obtener todos los registros de la colección
for doc in mycollection.find():
    print(doc)

{'_id': ObjectId('66d511a6fd25f223afbb0a8c'), 'Fecha': '12/24/2023', 'sCampaign': 'CursosDeProgramacion', 'Region': 'Oeste', 'Clicks': 26148.0, 'Impresiones': 133468.0, 'Views': 21979.0, 'Costo': 606.0}
{'_id': ObjectId('66d511a6fd25f223afbb0a8d'), 'Fecha': '3/12/2024', 'sCampaign': 'CursosDeProgramacion', 'Region': 'Oeste', 'Clicks': 32254.0, 'Impresiones': 206296.0, 'Views': 6448.0, 'Costo': 726.0}
{'_id': ObjectId('66d511a6fd25f223afbb0a8e'), 'Fecha': '3/20/2024', 'sCampaign': 'CursosDeProgramacion', 'Region': 'Norte', 'Clicks': 7761.0, 'Impresiones': 178949.0, 'Views': 47475.0, 'Costo': 862.0}
{'_id': ObjectId('66d511a6fd25f223afbb0a8f'), 'Fecha': '10/22/2023', 'sCampaign': 'CursosDeProgramacion', 'Region': 'Este', 'Clicks': 42967.0, 'Impresiones': 356762.0, 'Views': 7496.0, 'Costo': 1722.0}
{'_id': ObjectId('66d511a6fd25f223afbb0a90'), 'Fecha': '4/22/2024', 'sCampaign': 'CursosDeProgramacion', 'Region': 'Norte', 'Clicks': 13362.0, 'Impresiones': 157531.0, 'Views': 47034.0, 'Costo'

In [40]:
def read_collection_df(collection):
    return pd.DataFrame(list(collection.find()))

read_collection_df(mycollection)

,_id,Fecha,sCampaign,Region,Clicks,Impresiones,Views,Costo
0,66d511a6fd25f223afbb0a8c,12/24/2023,CursosDeProgramacion,Oeste,26148.0,133468.0,21979.0,606.0
1,66d511a6fd25f223afbb0a8d,3/12/2024,CursosDeProgramacion,Oeste,32254.0,206296.0,6448.0,726.0
2,66d511a6fd25f223afbb0a8e,3/20/2024,CursosDeProgramacion,Norte,7761.0,178949.0,47475.0,862.0
3,66d511a6fd25f223afbb0a8f,10/22/2023,CursosDeProgramacion,Este,42967.0,356762.0,7496.0,1722.0
4,66d511a6fd25f223afbb0a90,4/22/2024,CursosDeProgramacion,Norte,13362.0,157531.0,47034.0,517.0
...,...,...,...,...,...,...,...,...
92,66d511a6fd25f223afbb0ae8,4/1/2024,CursosDeProgramacion,Oeste,29805.0,381302.0,19833.0,1152.0
93,66d511a6fd25f223afbb0ae9,5/12/2024,CursosDeProgramacion,Sur,49996.0,79199.0,38665.0,1603.0
94,66d511a6fd25f223afbb0aea,5/7/2024,CursosDeProgramacion,Este,22068.0,435276.0,34668.0,577.0
95,66d511a6fd25f223afbb0aeb,10/31/2023,CursosDeProgramacion,Este,31943.0,248545.0,21555.0,882.0


La línea de código `pd.DataFrame(list(collection.find()))` está realizando varias operaciones para convertir los documentos de una colección de MongoDB en un DataFrame de Pandas. Aquí está la explicación detallada de cómo funciona:

1. **`collection.find()`**:
   - `collection` es un objeto que representa una colección en una base de datos MongoDB.
   - El método `find()` se utiliza para realizar una consulta en la colección y recuperar todos los documentos. Si no se pasan parámetros a `find()`, se recuperan todos los documentos de la colección.
   - El resultado de `find()` es un cursor, que es un iterador que permite recorrer los documentos recuperados uno por uno.

2. **`list(collection.find())`**:
   - La función `list()` toma el cursor devuelto por `find()` y lo convierte en una lista de documentos. Cada documento en la lista es un diccionario de Python que representa un documento de MongoDB.
   - Esta conversión es necesaria porque el cursor no puede ser directamente pasado a `pd.DataFrame()`. La lista resultante contiene todos los documentos de la colección en forma de diccionarios.

3. **`pd.DataFrame(list(collection.find()))`**:
   - `pd.DataFrame()` es una función de la biblioteca Pandas que se utiliza para crear un DataFrame.
   - Al pasar la lista de diccionarios a `pd.DataFrame()`, Pandas convierte cada diccionario en una fila del DataFrame. Las claves de los diccionarios se convierten en los nombres de las columnas del DataFrame.
   - El resultado es un DataFrame de Pandas que contiene todos los documentos de la colección de MongoDB, con las columnas correspondientes a las claves de los diccionarios.

En resumen, esta línea de código recupera todos los documentos de una colección de MongoDB, los convierte en una lista de diccionarios y luego crea un DataFrame de Pandas a partir de esa lista. Esto permite trabajar con los datos de MongoDB utilizando las poderosas herramientas de análisis y manipulación de datos que ofrece Pandas.

#### Ejecución de consultas para extraer datos específicos en MongoDB

In [41]:
def read_collection_df(collection, query={}):
    return pd.DataFrame(list(collection.find(query)))

query = {'Region': 'Norte'}
read_collection_df(mycollection, query)

,_id,Fecha,sCampaign,Region,Clicks,Impresiones,Views,Costo
0,66d511a6fd25f223afbb0a8e,3/20/2024,CursosDeProgramacion,Norte,7761.0,178949.0,47475.0,862.0
1,66d511a6fd25f223afbb0a90,4/22/2024,CursosDeProgramacion,Norte,13362.0,157531.0,47034.0,517.0
2,66d511a6fd25f223afbb0a9a,4/6/2024,CursosDeProgramacion,Norte,23294.0,282996.0,24528.0,239.0
3,66d511a6fd25f223afbb0a9f,3/3/2024,CursosDeProgramacion,Norte,20211.0,50727.0,10991.0,573.0
4,66d511a6fd25f223afbb0aac,1/26/2024,CursosDeProgramacion,Norte,42772.0,203801.0,37980.0,1110.0
5,66d511a6fd25f223afbb0aad,4/8/2024,CursosDeProgramacion,Norte,48174.0,61418.0,5641.0,269.0
6,66d511a6fd25f223afbb0aae,7/14/2024,CursosDeProgramacion,Norte,22114.0,247486.0,23409.0,1604.0
7,66d511a6fd25f223afbb0ab1,10/6/2023,CursosDeProgramacion,Norte,20895.0,391347.0,15374.0,1533.0
8,66d511a6fd25f223afbb0ab5,11/26/2023,CursosDeProgramacion,Norte,17816.0,386508.0,42831.0,1281.0
9,66d511a6fd25f223afbb0abb,11/7/2023,CursosDeProgramacion,Norte,28533.0,169646.0,25505.0,1895.0
